#### Read data into numpy array

In [ ]:
from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets


uploader = widgets.FileUpload(
    accept='.EDE',  # Accepted file extension e.g. '.txt', '.pdf', 'image/*', 'image/*,.pdf'
    multiple=False  # True to accept multiple files upload else False
)
display(uploader)

In [ ]:
import io
import numpy as np


uploaded_file = next(iter(uploader.value.values()))
stream = io.BytesIO(uploaded_file["content"])
data = np.frombuffer(stream.getbuffer(), dtype=np.uint16)
data = data.reshape(256,256)

print("Total data size:", data.size)
print("Data value boundaries: min = {}\tmax = {}".format(np.min(data), np.max(data)))

In [ ]:
## Read data from file into numpy array
# import numpy as np


# data_file = 'spectrum/108_DATA.EDE'
# with open(data_file, "rb") as f:
#     data = np.fromfile(f, dtype=np.uint16)
# data = data.reshape(256,256)

# print("Total data size:", data.size)
# print("Data value boundaries: min = {}\tmax = {}".format(np.min(data), np.max(data)))

In [ ]:
import plotly.graph_objects as go

fig_3d = go.Figure(data=[go.Surface(z=data, colorscale='Greys', cmin=0, cmax=2*np.max(data))])
fig_3d.update_layout(title='Three-dimensional surface of dE-E spectrum', title_x=0.5, autosize=True)

# Ref.: https://plotly.com/python/3d-camera-controls/
camera = dict(
    up=dict(x=0, y=0, z=1),
    center=dict(x=0, y=0, z=0),
    eye=dict(x=1.25, y=1.25, z=1.25)
)

fig_3d.update_layout(scene_camera=camera)

g_3d = go.FigureWidget(fig_3d)

def remove_noise(change):
    d = data.copy()
    d[data <= threshold_3d.value] = 0
    with g_3d.batch_update():
        g_3d.data[0].z = d

threshold_3d = widgets.IntSlider(
    min=0, max=np.max(data),
    description='Noise threshold:',
    continuous_update=False
)

threshold_3d.observe(remove_noise, names="value")

widgets.VBox([threshold_3d,
              g_3d])

In [ ]:
from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

import plotly.graph_objects as go
from plotly.subplots import make_subplots


def get_2d_data(data, threshold = 0):
    sx = []
    sy = []
    sc = []
    idx = 0
    n,m = data.shape
    for y in range(n):
        for x in range(m):
            if data[y][x] > threshold:
                sx.append(x)
                sy.append(y)
                sc.append(data[y][x])
                idx += 1
    return sx, sy, sc

sx, sy, sc = get_2d_data(data, 1)
sx = np.array(sx)
sy = np.array(sy)
sc = np.array(sc)

# Initialize figure with subplots
fig = make_subplots(
    rows=2, cols=2,
    column_widths=[0.6, 0.4],
    row_heights=[0.4, 0.6],
    specs=[[{"type": "scattergl", "rowspan": 2}, {"type": "scattergl"}],
           [            None                   , {"type": "scattergl"}]],
    subplot_titles=("Two-dimensional dE-E spectrum",
                    "E detector (Max)",
                    "dE detector (Max)")
)

fig.add_trace(
    go.Scattergl(
        x = sx,
        y = sy,
        # opacity=1.,
        showlegend=False,
        mode='markers',
        marker=dict(
            size=2,
            cmin=0,
            cmax=np.max(sc),
            color=sc,
            colorscale='Gray_r',
            showscale=True,
            # colorbar=dict(x=-0.1)
            # opacity=1.
        )
    ),
    row=1, col=1
)

# Data for plotting
e = np.arange(256)
y1 = np.amax(data, axis=1)
y2 = np.amax(data, axis=0)

fig.add_trace(
    go.Scattergl(
        x = e,
        y = y1,
        showlegend=False,
        marker=dict(color="black"),
        # name="Projection"
    ),
    row=1, col=2
)

fig.add_trace(
    go.Scattergl(
        x = e,
        y = y2,
        showlegend=False,
        marker=dict(color="black")
    ),
    row=2, col=2
)

# Update subplot properties
fig.update_layout(
    # title='Two-dimensional dE-E spectrum', title_x=0.5,
    plot_bgcolor='white',
    xaxis = dict(range=[0,255]),
    yaxis = dict(range=[0,255]),
)

g = go.FigureWidget(fig)

# create click callback function
def update_point(trace, points, selector):
    if trace is not g.data[0]:
        return
    
    d = np.zeros_like(data)
    for i in points.point_inds:
        x, y = trace.x[i], trace.y[i]
        d[y,x] = data[y,x]
    
    with g.batch_update():
        g.data[1].y = np.amax(d, axis=1)
        g.data[2].y = np.amax(d, axis=0)

g.layout.clickmode = 'event+select'
g.data[0].on_selection(update_point)

def validate():
    return True

def response(change):
    if validate():
        d = data.copy()
        d[data <= threshold.value] = 0

        e = np.arange(256)
        y1 = np.amax(d, axis=1)
        y2 = np.amax(d, axis=0)

        with g.batch_update():
            _sx, _sy, _sc = get_2d_data(d)
            g.data[0].x = _sx
            g.data[0].y = _sy
            g.data[0].marker.color = _sc
            g.data[1].y = y1
            g.data[2].y = y2

g.layout.xaxis.title = 'Energy of E detector (Channel no.)'
g.layout.yaxis.title = 'Energy of dE detector (Channel no.)'

g.layout.yaxis2.title = 'Intensity (a.u.)'
g.layout.xaxis3.title = 'Binding energy (eV)'
g.layout.yaxis3.title = 'Intensity (a.u.)'

# g.data[0].selected.marker.color='blue'
g.data[0].unselected.marker.opacity=1.

threshold = widgets.IntSlider(
    min=0, max=np.max(data),
    step=1, value=1,
    description='Noise threshold:',
    continuous_update=False
)

threshold.observe(response, names="value")

widgets.VBox([threshold,
              g])

In [ ]:
# import plotly.graph_objects as go

# fig = go.Figure(data=go.Heatmap(z=data, colorscale='Greys'))
# fig.update_layout(
#     title='Two-dimensional dE-E spectrum', title_x=0.5,
#     autosize=False,
#     width=500, height=500,
# )
# fig.show()

In [ ]:
# import plotly.graph_objects as go

# fig = go.Figure(data =
#      go.Contour(
#         z=data,
#         colorscale='Greys',
#     ))
# fig.show()